The purpose of this notebook is to maintain the kiva module that provides a complete and cleaner data set. This will allow the team to streamline the call to our database to run our analysis.   

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from config import dbconfig

In [2]:
engine = create_engine(dbconfig.aws_connection_string)

In [3]:
#creating full_loans data set from database
days = pd.read_sql_query("SELECT * FROM loan_dates", engine)
loans = pd.read_sql_query("SELECT * FROM loans", engine)
partners = pd.read_sql_query("SELECT * FROM partners", engine)
loans_days = pd.merge(loans, days, how = 'left', on = 'loan_id')
partners = partners.rename(columns = {"id": "partner_id"})
full_loans = pd.merge(loans_days, partners, how = "left", on = "partner_id", suffixes=("_loans", "_partners"))

In [4]:
# creating a subset only with field partner loans
fl_subset = full_loans[full_loans.distribution_model == "field_partner"]

#creating a new binary variable that is 0 if the loan was successfully funded and 1 otherwise
fl_subset["status_loans_bi"] = np.where(fl_subset["status_loans"] == "funded", 0, 1)

#Cleaning data set to select only funded loans and get rid of negative values in posted_to_raised_days
fl_subset_cl = fl_subset[fl_subset.status_loans_bi == 0]
fl_subset_cl = fl_subset_cl[fl_subset_cl.posted_to_raised_days >= 0]

/Users/yavah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [5]:
#fixing the format of the dates columns
fl_subset_cl['posted_time_actual'] = fl_subset_cl['posted_time_actual'].astype('int64')
fl_subset_cl['disburse_time_actual'] = fl_subset_cl['disburse_time_actual'].astype('int64')
fl_subset_cl['raised_time_actual'] = fl_subset_cl['raised_time_actual'].astype('int64')

In [8]:
#Saving table to database
fl_subset_cl.to_sql('kiva_data_flat', engine, if_exists='replace', chunksize = 10000)

In [14]:
print(engine.table_names())

['partners', 'loan_dates', 'd_date', 'lenders', 'loans', 'loans_lenders', 'kiva_data_flat']
